In [49]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [62]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [50]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  


default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']


default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']


Columns_Data_ZSD10_Exito_Cencosub_ZVMI=['Fecha doc.', 'Solicitante', 'Cliente', 'Nº de cliente 1', 'Marca',
       'Material', 'Número de material', 'Unidades', 'UM', 'ValorNeto',
       'Nº pedido cliente', 'Doc.venta', 'ClVt', 'Solic.', 'OrgVt', 'CDis',
       'OfVta', 'GVen', 'Organización de ventas', 'Mon.', 'Jquía.productos',
       'Total Pedidos', 'Plataforma', 'Fe.PedCpas', 'Creado por']

Columns_Data_ZSD038_Exito_Cencosub_Rechazados=['Doc.comer.', 'Fecha doc.', 'Destinat.', 'Nombre 1', 'Cód.Vend.',
       'Nombre vendedor', 'MR', 'Denominación', 'Material', 'Denominación.1',
       'Marca', 'Categoría', 'Ctd.confirmada', 'Valor neto', 'Mon.', 'CDis',
       'Solic.', 'Nombre']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Dta Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)

In [51]:
"""""-------------------------------------------Limpio los datos-----------------------------------"""

# Esta session combierte los valores monetarios en enteros str a int para posteriores calculos
Canal01_Pendiente['Vlr.Neto P'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])

# Esta session combierte colunmas de float .0 a enteros int, tambien recibe valores Null
Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1']=Funtions.Estandarizo_Pedidos(Canal01_Pendiente['Pedido.1'],Canal01_Entregado['Pedido.1'])


# ¿Preguntar cantidad pendiente P y C significado

In [52]:
"""Seccion para coompletar los valores necesarios para trabajar"""

Agenda=["85","20","146","149","50","138","45"]
# En este caso solo aplica para Entrega y pendiente en el cual completa con 00 los pedidos del Exito
Canal01_Entregado['Pedido.1']=Funtions.complete_pedidos(Canal01_Entregado['Pedido.1'],Agenda)
Canal01_Pendiente['Pedido.1']=Funtions.complete_pedidos(Canal01_Pendiente['Pedido.1'],Agenda)

In [53]:
"""---------------------------------------Unifico los dataframes de pedido y entregas--------------------------------------------------------------"""
Default_Column_Informe=['Canal','Denomin.','Solic.','Nombre','Pedido','Clase Pedido','Material','Valor Neto','Cantidad']

longuitud_Pendiente=len(Canal01_Pendiente)
longuitud_Entrega=len(Canal01_Entregado)


Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


Canal01_Pendiente_Informe=Canal01_Pendiente[['Can.distr.','Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Vlr.Neto P','Cantidad P']]
Canal01_Entregado_Informe=Canal01_Entregado[['Can.distr.', 'Denomin.','Solic.','Nombre','Pedido.1','Clase Pedi','Material','Valor Neto','Cantidad E']]


print(longuitud_Pendiente,longuitud_Entrega)


"""---En esta sección analizar si borro duplicados-----"""

Canal01_Entregado_Pendiente_Informe=pd.concat([Funtions.default_column(Default_Column_Informe,Canal01_Pendiente_Informe),Funtions.default_column(Default_Column_Informe,Canal01_Entregado_Informe)]).reset_index(drop=True).drop_duplicates()

759 3047


In [38]:
""""------------------------------------ Estandarizo datos de columnas del informe de las lineas que deberian ser numericos------------------------------------------------"""

try:
    Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad']=Funtions.Clean_Num_List(Canal01_Entregado_Pendiente_Informe['Valor Neto'],Canal01_Entregado_Pendiente_Informe['Cantidad'])
except:
    print("intento")
    pass


,Canal,Denomin.,Solic.,Nombre,Pedido,Clase Pedido,Material,Valor Neto,Cantidad
0,1,MODERNO,1000000000,ALMACENES EXITO SA,0146323487,ZVMI,PB0088954,5040.0,1.0
7,1,MODERNO,1000000000,ALMACENES EXITO SA,0146323487,ZVMI,3004290300,0.0,1.0
10,1,MODERNO,1000000000,ALMACENES EXITO SA,0146323487,ZVMI,PB0068847,0.0,1.0
14,1,MODERNO,1000000000,ALMACENES EXITO SA,0146323487,ZVMI,PB0074055,0.0,1.0
25,1,MODERNO,1000000000,ALMACENES EXITO SA,0146323487,ZVMI,PB0077985,0.0,1.0
...,...,...,...,...,...,...,...,...,...
3046,1,MODERNO,1000001609,FAMATODO CENTRO DE DISTRIBUCION,6011065,ZED1,PB0086371,383962.0,36.0
3047,1,MODERNO,1000001609,FAMATODO CENTRO DE DISTRIBUCION,6011065,ZED1,PB0093442,3018127.0,264.0
3048,1,MODERNO,1000001609,FAMATODO CENTRO DE DISTRIBUCION,6011065,ZED1,PB0093512,3558548.0,280.0
3049,1,MODERNO,1000001609,FAMATODO CENTRO DE DISTRIBUCION,6011065,ZED1,PB0081191,423105.0,25.0


In [54]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito Entrega----------------------------------"""
Filtro_Agenda_Exito_Entrega_Pendiente=Funtions.Search_Agenda_Exito(Canal01_Entregado_Pendiente_Informe['Pedido'],Agenda_Exito)
Filtro_Agenda_Cencosub_Entrega_Pendiente=Funtions.Search_agenda_Cencosub(Canal01_Entregado_Pendiente_Informe["Pedido"],Agenda_Cencosub)

In [55]:
"""-------------------------------------------------Aplico filtro para el consolidados------------------------------------------------------------"""


"""------------------------------Canal 01 Entrega con y sin agenda Exito-----------------------"""
#Agenda Exito
Filtro_Canal01_Entregado_Pendiente_Exito=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Exito_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Exito=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Exito].reset_index(drop=True)

#Agenda Censosud
"""------------------------------Canal 01 Entrega con y sin agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Pendiente_Cencosub=(Canal01_Entregado_Pendiente_Informe['Pedido'].isin(Filtro_Agenda_Cencosub_Entrega_Pendiente))   #Excluyo con ~
Canal01_Entregado_Pendiente_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)
Canal01_Entregado_Pendiente_Sin_Agenda_Cencosub=Canal01_Entregado_Pendiente_Informe.loc[~Filtro_Canal01_Entregado_Pendiente_Cencosub].reset_index(drop=True)


In [56]:
"""-------------------------------------------------------------Crea las tablas dinamicas--------------------------------------------------------"""
Tablas_Creadas=[]

# Tabla dinácamica Exito
try:
    Table_dinamica_Exito_entrega_pendiente=Canal01_Entregado_Pendiente_Agenda_Exito.pivot_table(index=['Pedido','Clase Pedido'],aggfunc={'Material':'count','Cantidad':sum,'Valor Neto':sum}).reset_index()
    Lineas_dinamica_Exito_entrega_pendiente=sum(Table_dinamica_Exito_entrega_pendiente["Material"])
    Tablas_Creadas.append("Lineas_dinamica_Exito_entrega_pendiente")
except:
    pass


# Tabla dinácamica Exito Cencosud
try:
    Table_dinamica_Cencosub_entrega_pendiente=Canal01_Entregado_Pendiente_Agenda_Cencosub.pivot_table(index=['Pedido','Clase Pedido'],aggfunc={'Material':'count','Cantidad':sum,'Valor Neto':sum}).reset_index()
    Lineas_dinamica_Cencosub_entrega_pendiente=sum(Table_dinamica_Cencosub_entrega_pendiente["Material"])
    Tablas_Creadas.append("Table_dinamica_Cencosub_entrega_pendiente")
except:
    pass

In [60]:
Table_dinamica_Exito_entrega_pendiente

,Pedido,Clase Pedido,Cantidad,Material,Valor Neto
0,0020657846,ZVMI,1.0,1,13107.0
1,0050443107,ZED1,30.0,1,537690.0
2,0050443312,ZED1,22.0,2,895510.0
3,0050443367,ZED1,12.0,2,505608.0
4,0050444502,ZVMI,1.0,1,13107.0
5,0085764470,ZEDI,23.0,11,440657.0
6,0085764471,ZEDI,68.0,14,2892464.0
7,0085765393,ZEDI,18.0,2,507828.0
8,0085768022,ZED1,144.0,4,7631850.0
9,0085768075,ZVMI,1952.0,1083,23459213.0


In [63]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Exito_entrega_pendiente['Pedido'],Table_dinamica_Cencosub_entrega_pendiente['Pedido']])

#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [65]:
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [66]:
# Utilizo la transaccion para buscar ZSD037 en entrega para las fechas
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD037_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD037_Pedidos_Exito_Cencosub)
Data_ZSD037_Pedidos_Exito_Cencosub=Data_ZSD037_Pedidos_Exito_Cencosub[Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]


# Ver metodo estandarizar pedidos

In [67]:
try:
    Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']=SAP_GUI.Estandarizo_Pedidos(Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'])   #OrgVt   #CDis  #4940
except:
    try:
        Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']=Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'].astype(int)
    except:
        None

In [68]:
#Funtions.complete_pedidos(Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'],Agenda)

Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']


0       0020657846
1       0020657846
2       0020657846
3       0020657846
4       0020657846
           ...    
3673    60-1164151
3674    60-1164151
3675    60-1164151
3676    60-1164151
3677    60-1164151
Name: Nº pedido cliente, Length: 3678, dtype: object

In [75]:
def Complete_00(valor):
    while len(str(valor))<10:
        valor="0"+valor
    return(valor)


def complete_pedidos(Dataset,Agenda):
     """
     -Dataset: Entrego columna 
     -Agenda: Agenda con los valores a encontrar
     (Requiere funcion Complete_00)
     """
     lista=[]
     Agenda='^'+("|^").join(Agenda)
     for i in Dataset:
          if re.findall(f'({Agenda})',str(i))!=[]:
               # Encuentra longuitud de 
               if len(str(i))>=8 and len(str(i))<=9:
                    i=Complete_00(str(i))
                    lista.append(str(i))
               else:
                    lista.append(str(i))
          else:
               lista.append(str(i))
     return(lista)

In [76]:
Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente']=complete_pedidos(Data_ZSD037_Pedidos_Exito_Cencosub['Nº pedido cliente'],Agenda)

In [78]:
#Dataframe de interes
Data_ZSD037_Pedidos_Pendientes_Fecha=Data_ZSD037_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()

In [79]:
Data_ZSD037_Pedidos_Pendientes_Fecha

,Nº pedido cliente,PrimFeEntr,ÚltEntrega
0,0020657846,27.06.2023,04.07.2023
20,0050443107,01.07.2023,01.07.2023
30,0050444502,27.06.2023,08.07.2023
40,0085764470,29.06.2023,29.06.2023
49,0050443312,08.07.2023,08.07.2023
59,0050443367,08.07.2023,08.07.2023
147,0085764471,29.06.2023,29.06.2023
247,0085765393,24.06.2023,28.06.2023
256,0085768022,08.07.2023,08.07.2023
266,0085768075,29.06.2023,29.06.2023


In [81]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [82]:
Tabla_Agenda_Exito

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedido,Cantidad,Material,Valor Neto,Plataforma
0,0020657846,27.06.2023,04.07.2023,ZVMI,1.0,1,13107.0,VEGAS
1,0050443107,01.07.2023,01.07.2023,ZED1,30.0,1,537690.0,CALI
2,0050443312,08.07.2023,08.07.2023,ZED1,22.0,2,895510.0,CALI
3,0050443367,08.07.2023,08.07.2023,ZED1,12.0,2,505608.0,CALI
4,0050444502,27.06.2023,08.07.2023,ZVMI,1.0,1,13107.0,CALI
5,0085764470,29.06.2023,29.06.2023,ZEDI,23.0,11,440657.0,FUNZA
6,0085764471,29.06.2023,29.06.2023,ZEDI,68.0,14,2892464.0,FUNZA
7,0085765393,24.06.2023,28.06.2023,ZEDI,18.0,2,507828.0,FUNZA
8,0085768022,08.07.2023,08.07.2023,ZED1,144.0,4,7631850.0,FUNZA
9,0085768075,29.06.2023,29.06.2023,ZVMI,1952.0,1083,23459213.0,FUNZA


In [88]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD037_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega_pendiente,how="right",left_on="Nº pedido cliente",right_on="Pedido")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedido', 'Cantidad', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [89]:
Tabla_Agenda_Exito

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedido,Cantidad,Material,Valor Neto,Plataforma
0,60-1164151,01.07.2023,08.07.2023,ZEDI,118.0,9,2427376.0,60-1164151
